# Preprocess JSN-data
Cropping bilateral images into left and right knee (ROI)

Preprocess images (resize, normalize, gaussian blur, clahe)

Standardize numeric data with Standard Scaler and MinMax normalization



author = MV<br>
date = 2021-10-19<br>

_______________________________________

- JSN_pred_clf.csv
    - class def: 10% JSN
    - exclusion: KL0-0, KL1-1, KL4 at baseline
    - with pre-processed img path (img_path)
    
- JSN_pred_clf_new.csv
    - class def: 10% JSN
    - exclusion: KL0-0, KL1-1, KL4 at baseline
    
- JSN_pred_clf_pro.csv
    - class def: 10% JSN
    - exclusion: KL0-0, KL1-1, KL4 at baseline, implant imgs
    - with pre-processed img path (img_path_pro)
    
- /srv/Class_def_files/JSN_final_pro.csv
    - class def: 20% JSN
    - exclusion: KL4 at baseline, implant imgs
    - with pre-processed img path (img_path_pro)
    
- /srv/Normalized_numeric_data/StandardScaler_Master_20JSN_XML_ex014.csv
    - Standard Scaler normalized data of  Master_20JSN_XML_ex014.csv
    
- /srv/Normalized_numeric_data/MinMax_Master_20JSN_XML_ex014.csv
    - MinMax normalized data of  Master_20JSN_XML_ex014.csv
    
- /srv/Normalized_numeric_data/StandardScaler_Master_20JSN_XML_ex4.csv
    - Standard Scaler normalized data of  Master_20JSN_XML_ex4.csv
    
- /srv/Normalized_numeric_data/MinMax_Master_20JSN_XML_ex4.csv
    - MinMax normalized data of  Master_20JSN_XML_ex4.csv

# Imports

In [ ]:
! nvidia-smi

In [ ]:
# this defines the GPU you are using
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
# add paths for dnn2 and labelbox-connector
import sys
sys.path.insert(1, "/srv/dnn-framework2")

sys.path.insert(1, "/srv/labelbox-connector")

In [ ]:
#general
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import logging
import cv2
import copy

#tensorflow
import tensorflow as tf


import shutil
import os

import pydicom

from lxml import etree
import xml.etree.ElementTree as ET
from framework.data_objects import BoundingBox, PointList2D, DicomImage
import ast
from framework.inferences import Inference

In [ ]:
#general
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import logging
import cv2
import copy

#tensorflow
import tensorflow as tf
from tensorflow import keras

from PIL import Image
import ast

In [ ]:
#configs
%matplotlib inline
logging.basicConfig(format='%(asc' 'time)s %(name)-25s %(level' 'name)-8s %(message)s')
logging.getLogger().setLevel(logging.INFO) # you change this to logging.DEBUG to get more logging information

# Crop images

## Crop imgs of data with exclusion of KL0-0, KL1-1 and KL4 baseline (10%JSN)

In [ ]:
## Functions to crop image ROI if you have the KOALA output for the images

def fake_jsw_box(xml_path, side="R"):
    xm = etree.parse(xml_path)
    region_id = 0 if side=="R" else 1
    ptlist = []
    for loc in {'LAT','MED'}:
        #loc = 'LAT' if i==3 else 'MED'
        for k in range(4):
            xpt = f"/Analysis/Region[id={region_id}]/MeasurementBoard[id='JSx']/JSx[location='{loc}']/JSW{k}/coordinates"
            if len(xpt) == 0:
                continue
            xy = ast.literal_eval(xm.xpath(xpt)[0].text.replace(" ",","))
            coords = [ xy[i:i + 2] for i in range(0,len(xy),2)]
            ptlist.extend(coords)

    ptlist = PointList2D(ptlist)

    c = ptlist.center.data
    dx = int(1.6 * ptlist.dx)
    dy = int(2. * dx)
    #print(c, dx, dy)
    return BoundingBox.from_center_height_width(c, dy, dx)


def cut_img_box(df_in, loc):
    l = df_in.loc[loc]['Laterality']
    bbox = fake_jsw_box("/mnt/temporaer/MV/KOALA_output/xml/"+df_in.loc[loc]['ID']+".xml", l)
    return bbox

## Crop imgs of data with exclusion of only KL4 baseline ( 20% JSN)

In [ ]:
#load csv file with dicom image path

df=pd.read_csv('/srv/Class_def_files/JSN_final.csv')

In [ ]:
## Add dicom path 
df['dicom_img_path']=''
for i, row in df.iterrows():
    print(i, end='\r')  
    if df.Study[i] == 'OAI':
        df['dicom_img_path'][i] = '/mnt/fs/37_OAI/data/dicom/' +  df['ID'][i] + '.dcm'
    elif df.Study[i] == 'MOST':
        df['dicom_img_path'][i] = '/mnt/fs/40_MOST/data/dicom/' +  df['ID'][i] + '.dcm'
    else:
        df['dicom_img_path'][i] = '/mnt/fs/102_CHECK/data/dicom/' +  df['ID'][i] + '.dcm'

In [ ]:
# cut img and write in folder

compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 0, cv2.IMWRITE_PNG_STRATEGY, cv2.IMWRITE_PNG_STRATEGY_FIXED]
maxval = 20000
clahe = cv2.createCLAHE(40, (15,15))

for i, row in df.iterrows():
    i = i+36820
    print(i, end='\r')
    
    # cut img
    l = df.loc[i]['Laterality']
    fn = os.path.join('/mnt/caroline/MV/Cropped_knees', df.loc[i]['ID']+'_'+ l + '.png')
    
    bbox = fake_jsw_box("/mnt/temporaer/MV/KOALA_output/xml/"+df.loc[i]['ID']+".xml", l)
    
    dicom_path = df.dicom_img_path[i] 
    dicom_image = DicomImage(dicom_path).load()
    img = pydicom.dcmread(dicom_path).pixel_array

    cut_image = bbox.cut(img)

    # flip if left knee
    if l == 'L':
        cut_image = np.fliplr(cut_image)

    # write image on caroline
    cv2.imwrite(fn,cut_image, compression_params)
    
    # write cropped img path in dataframe 
    df['img_path'][i]=fn
    
    ## pre-process img
    img_p = cv2.resize(cut_image, (512,1024), interpolation=cv2.INTER_NEAREST)
    img_p = cv2.normalize(img_p, img_p,0,maxval,cv2.NORM_MINMAX ,cv2.CV_16UC1)
    img_p = cv2.GaussianBlur(img_p,(3,3), 0.3)
    img_p = clahe.apply(img_p)

    # write pre-processed image on caroline (location you want to store images. Same location of the training notebook)
    fn_p = os.path.join('/mnt/caroline/MV/Cropped_knees/Clahe-pngs', df.loc[i]['ID']+'_'+ l + '.png')
    cv2.imwrite(fn_p,img_p, compression_params)
    
    # write image path of the cropped image in dataframe 
    df['img_path_pro'][i]=fn_p

In [ ]:
# write to csv file
df.to_csv('/srv/Class_def_files/JSN_final_pro.csv')

## Crop residual imgs of Masterfiles

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex4.csv')
len(df)

In [ ]:
df = df.rename(columns={'img_path':'dicom_img_path'})

# cut img and write in folder

compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 0, cv2.IMWRITE_PNG_STRATEGY, cv2.IMWRITE_PNG_STRATEGY_FIXED]
maxval = 20000
clahe = cv2.createCLAHE(40, (15,15))

indices=[]

for i, row in df.iterrows():
    i = i
    print(i, end='\r')
    
    try:
        test = cv2.imread(df.img_path_pro[i])
        plt.imshow(test)
    except:
        # cut img
        l = df.loc[i]['Laterality']
        fn = os.path.join('/mnt/caroline/MV/Cropped_knees', df.loc[i]['ID']+'_'+ l + '.png')

        bbox = fake_jsw_box("/mnt/temporaer/MV/KOALA_output/xml/"+df.loc[i]['ID']+".xml", l)

        dicom_path = df.dicom_img_path[i] 
        dicom_image = DicomImage(dicom_path).load()
        img = pydicom.dcmread(dicom_path).pixel_array

        cut_image = bbox.cut(img)

        # flip if left knee
        if l == 'L':
            cut_image = np.fliplr(cut_image)

        # write image on caroline
        cv2.imwrite(fn,cut_image, compression_params)

        # write path in dataframe 
        df['img_path_cut'][i]=fn

        ## pre-process img
        img_p = cv2.resize(cut_image, (512,1024), interpolation=cv2.INTER_NEAREST)
        img_p = cv2.normalize(img_p, img_p,0,maxval,cv2.NORM_MINMAX ,cv2.CV_16UC1)
        img_p = cv2.GaussianBlur(img_p,(3,3), 0.3)
        img_p = clahe.apply(img_p)

        fn_p = os.path.join('/mnt/caroline/MV/Cropped_knees/Clahe-pngs', df.loc[i]['ID']+'_'+ l + '.png')
        cv2.imwrite(fn_p,img_p, compression_params)

        # write path in dataframe 
        df['img_path_pro'][i]=fn_p
    

# Split into Training, tune and test set

In [ ]:
## Splitting of data 10%JSN

df = pd.read_csv('/srv/JSN_pred_clf_pro.csv')

np.random.seed(137115)
train_split = int(len(df) * 0.8)
test_split = len(df) - train_split

index_test = np.sort(np.random.choice(range(0, len(df)), size=test_split, replace=False))
index_train = np.delete(range(0, len(df)), index_test)

tune_split = int(len(index_train) * 0.15)
bla = np.sort(np.random.choice(range(0, len(index_train)), size=tune_split, replace=False))
bla2 = np.delete(range(0, len(index_train)), bla)
index_tune = index_train[bla]
index_train = index_train[bla2]

dfTrain = df.iloc[index_train]
dfTune = df.iloc[index_tune]
dfTest = df.iloc[index_test]


dfTrain.to_csv('/srv/JSN_pred_clf_pro_train.csv')
dfTest.to_csv('/srv/JSN_pred_clf_pro_test.csv')
dfTune.to_csv('/srv/JSN_pred_clf_pro_tune.csv')

In [ ]:
## splitting data of data 20% JSN and exclusion of only KL4 baseline

#df = pd.read_csv('/srv/Class_def_files/JSN_final_pro.csv')

np.random.seed(137115)
train_split = int(len(df) * 0.8)
test_split = len(df) - train_split

index_test = np.sort(np.random.choice(range(0, len(df)), size=test_split, replace=False))
index_train = np.delete(range(0, len(df)), index_test)

tune_split = int(len(index_train) * 0.15)
bla = np.sort(np.random.choice(range(0, len(index_train)), size=tune_split, replace=False))
bla2 = np.delete(range(0, len(index_train)), bla)
index_tune = index_train[bla]
index_train = index_train[bla2]

dfTrain = df.iloc[index_train]
dfTune = df.iloc[index_tune]
dfTest = df.iloc[index_test]


dfTrain.to_csv('/srv/Class_def_files/JSN_final_pro_train.csv')
dfTest.to_csv('/srv/Class_def_files/JSN_final_pro_test.csv')
dfTune.to_csv('/srv/Class_def_files/JSN_final_pro_tune.csv')

# Sort out manually implant images 
Implants.csv: indices of implant images

In [ ]:
df_test = pd.read_csv('/srv/JSN_pred_clf_pro.csv')
# All bilateral images --> inspection of only right images
df_R = df_test.loc[df_test.Laterality=='R']
df_new = df_R.reset_index()
df2 = df_new[7344:]
for i, row in df2.iterrows():
    print(i)
    plt.imshow(DicomImage(df2.dicom_img_path.loc[i]).load().array,cmap='gray')
    print(df2.ID[i])
    plt.show()
    

In [ ]:
## Eliminate Implants.csv indices from dataframe (438 images)

imp_df = pd.read_csv('/srv/Implants.csv')
orig_df = pd.read_csv('/srv/JSN_pred_clf_pro.csv')
orig_df = orig_df.drop(columns={'Unnamed: 0.1','Unnamed: 0.1.1','Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1'})
orig_df

imp_df = imp_df.replace(['l','r'], ['L','R'])
m = imp_df.merge(orig_df, on = ['Laterality', 'ID'], how = 'inner')


indices = []
for i, row in m.iterrows():
    idx = m['Unnamed: 0'][i]
    indices.append(idx)
    
df_new = orig_df.drop(indices, axis=0)

In [ ]:
df_new.to_csv('/srv/JSN_pred_clf_pro.csv')

# Normalize numeric data SKLEARN (StandardScaler & MinMaxScaler)

## Mastefile with exluscion KL 0,1,4

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex014.csv')


In [ ]:
from sklearn import preprocessing

## StandardScaler
col_names = ['Patient Sex','Patient Age','BMI','Hip_symptoms','WOMAC_dis','WOMAC_stiff','WOMAC_pain','other_knee_KOA','KL-grade_a','sclerosis_a','osteophytes_a']
new_col_names = ['Patient Sex_S','Patient Age_S','BMI_S','Hip_symptoms_S','WOMAC_dis_S','WOMAC_stiff_S','WOMAC_pain_S','other_knee_KOA_S','KL-grade_a_S','sclerosis_a_S','osteophytes_a_S']
features = df[col_names]

scaler = preprocessing.StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = new_col_names)

## eliminate all nan values
scaled_features = scaled_features.dropna()

scaled_features['ID'] = df.ID
scaled_features['Laterality'] = df.Laterality
scaled_features['class']= df['class']



In [ ]:
scaled_features.to_csv('/srv/Normalized_numeric_data/StandardScaler_Master_20JSN_XML_ex014.csv')

In [ ]:
## MinMaxScaler
col_names = ['Patient Sex','Patient Age','BMI','Hip_symptoms','WOMAC_dis','WOMAC_stiff','WOMAC_pain','other_knee_KOA','KL-grade_a','sclerosis_a','osteophytes_a']
new_col_names = ['Patient Sex_M','Patient Age_M','BMI_M','Hip_symptoms_M','WOMAC_dis_M','WOMAC_stiff_M','WOMAC_pain_M','other_knee_KOA_M','KL-grade_a_M','sclerosis_a_M','osteophytes_a_M']

features = df[col_names]

scaler = preprocessing.MinMaxScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = new_col_names)

## Eliminiate all nan values
scaled_features = scaled_features.dropna()

scaled_features['ID'] = df.ID
scaled_features['Laterality'] = df.Laterality
scaled_features['class']= df['class']

In [ ]:
scaled_features.to_csv('/srv/Normalized_numeric_data/MinMax_Master_20JSN_XML_ex014.csv')

### Masterfile with exclusion KL 4

In [ ]:
df = pd.read_csv('/srv/Class_def_files/Master_20JSN_XML_ex4.csv')


In [ ]:
from sklearn import preprocessing

## StandardScaler
col_names = ['Patient Sex','Patient Age','BMI','Hip_symptoms','WOMAC_dis','WOMAC_stiff','WOMAC_pain','other_knee_KOA','KL-grade_a','sclerosis_a','osteophytes_a']
new_col_names = ['Patient Sex_S','Patient Age_S','BMI_S','Hip_symptoms_S','WOMAC_dis_S','WOMAC_stiff_S','WOMAC_pain_S','other_knee_KOA_S','KL-grade_a_S','sclerosis_a_S','osteophytes_a_S']
features = df[col_names]


scaler = preprocessing.StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = new_col_names)

## eliminate all nan values
#scaled_features = scaled_features.dropna()
scaled_features['ID'] = df.ID
scaled_features['Laterality'] = df.Laterality
scaled_features['class']= df['class']

In [ ]:
scaled_features.to_csv('/srv/Normalized_numeric_data/StandardScaler_Master_20JSN_XML_ex4.csv')

In [ ]:
## MinMaxScaler
col_names = ['Patient Sex','Patient Age','BMI','Knee_inj','Hip_symptoms','WOMAC_dis','WOMAC_stiff','WOMAC_pain','other_knee_KOA','KL-grade_a','sclerosis_a','osteophytes_a']
features = df[col_names]

scaler = preprocessing.MinMaxScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)

## Eliminiate all nan values
scaled_features = scaled_features.dropna()

scaled_features['class']= df['class']

In [ ]:
scaled_features.to_csv('/srv/Normalized_numeric_data/MinMax_Master_20JSN_XML_ex4.csv')

# Testing for Parameter tuning for Pre-processing of images 

In [ ]:
def cut_img_box(df_in, loc):
    l = df_in.loc[loc]['Laterality']
    bbox = fake_jsw_box("/mnt/temporaer/MV/KOALA_output/xml/"+df_in.loc[loc]['ID']+".xml", l)
    return bbox

In [ ]:
df = pd.read_csv('/srv/JSN_pred_clf.csv')

In [ ]:
# output:
# 1. normalized, hist-equalized, 16 bit img
# 2. stats of histograms mean & std

In [ ]:
# gather info:
# 1. mix/max ranges in all images
# 2. good range for the 16bit pixel value range [0, 2^N -1]

In [ ]:
mins = []
maxes = []
windows = []
means = []
stds = []
global_hist = np.zeros((2**16 - 1),dtype=np.int64)

for i, row in df.iterrows():
    dcm = DicomImage(row['dicom_img_path']).load().array
    min_i = dcm.min()
    max_i = dcm.max()
    windows.append(max_i-min_i)
    maxes.append(max_i)
    mins.append(min_i)
    means.append(dcm.mean())
    stds.append(dcm.std())
    global_hist = np.histogram(dcm, (2**16 - 1))[0] + global_hist
    
    

In [ ]:
plt.yscale("log")
plt.plot(global_hist,)


In [ ]:
df["MIN"] = mins
df['MAX'] = maxes
df['window'] = windows
df['means'] = means
df['stds'] = stds

In [ ]:
indices = np.argwhere((means<30000)&(means>10000))
list1 = indices.tolist()

for idx in list1:
    df.drop(idx, inplace = True)

In [ ]:
indices2 = np.argwhere((means>60000))
list2 = indices2.tolist()

for idx in list2:
    df.drop(idx, inplace = True)

In [ ]:
# Add columns of Min, Max, widnow, means, stds to csv file
df.to_csv('/srv/JSN_pred_clf_pro.csv')

In [ ]:
plt.plot(df.window, 'r*')

In [ ]:
means = np.array(means)

maxval = 20000
clahe = cv2.createCLAHE(40, (20,20))

me = int(np.random.uniform()*df.shape[0])


In [ ]:
dcm = DicomImage(df.dicom_img_path.loc[100]).load()

img = None
img = cut_img_box(df, 100).cut(dcm.array)

img = cv2.resize(img, (512,1024), interpolation=cv2.INTER_NEAREST)
img = cv2.normalize(img, img,0,maxval,cv2.NORM_MINMAX,cv2.CV_16UC1)
#img = cv2.GaussianBlur(img,(3,3), 0.3)
img = cv2.equalizeHist(img)
#img = clahe.apply(img)

plt.figure(figsize=(8,13))
plt.imshow(img, cmap='gray')

